In [25]:
# Import our dependencies
%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sklearn as skl
import pandas as pd
import tensorflow as tf
import h5py
import keras
from keras import metrics

In [2]:
df = pd.read_csv("creditcardfraud_normalised.csv")
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
0,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,0.510600,...,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0.0
1,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,0.505267,...,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0.0
2,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,0.513018,...,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0.0
3,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,0.507585,...,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0.0
4,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,0.524303,...,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113137,0.841309,0.816024,0.792289,0.282615,0.740322,0.253078,0.251221,0.825094,0.481402,0.543023,...,0.562697,0.519669,0.662068,0.435184,0.647204,0.414671,0.421945,0.316362,0.001616,0.0
113138,0.945653,0.767278,0.844907,0.191801,0.760076,0.251339,0.270519,0.783252,0.483625,0.484598,...,0.561873,0.529193,0.672775,0.444037,0.567557,0.423245,0.412268,0.306299,0.005770,0.0
113139,0.979030,0.766959,0.841681,0.283484,0.764704,0.262167,0.265260,0.783963,0.479173,0.503229,...,0.559730,0.506699,0.661908,0.334122,0.618834,0.504274,0.416238,0.313345,0.001557,0.0
113140,0.935423,0.762401,0.853743,0.059288,0.762553,0.266011,0.263649,0.794324,0.480879,0.466926,...,0.560679,0.534024,0.667729,0.294862,0.568722,0.264905,0.423336,0.315146,0.002408,0.0


In [3]:
# Remove class target from features data
y = df["class"].values
X = df.drop(columns="class").values

In [62]:
Labels = df["class"].values
Features = df.drop(columns="class").values

In [4]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
number_input_features = len(X_train[0])

In [27]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR')
]

In [69]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=number_input_features))
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [70]:
# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 16)                480       
                                                                 
 dense_8 (Dense)             (None, 32)                544       
                                                                 
 dense_9 (Dense)             (None, 64)                2112      
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


In [71]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=METRICS)


In [72]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
2652/2652 [==============================] - 12s 3ms/step - loss: 0.0174 - tp: 64.0000 - fp: 234.0000 - tn: 112666.0000 - fn: 178.0000 - accuracy: 0.9964 - precision: 0.2148 - recall: 0.2645 - auc: 0.8055 - prc: 0.2753
Epoch 2/100
2652/2652 [==============================] - 10s 4ms/step - loss: 0.0044 - tp: 116.0000 - fp: 37.0000 - tn: 84652.0000 - fn: 51.0000 - accuracy: 0.9990 - precision: 0.7582 - recall: 0.6946 - auc: 0.9386 - prc: 0.6258
Epoch 3/100
2652/2652 [==============================] - 10s 4ms/step - loss: 0.0039 - tp: 129.0000 - fp: 38.0000 - tn: 84651.0000 - fn: 38.0000 - accuracy: 0.9991 - precision: 0.7725 - recall: 0.7725 - auc: 0.9423 - prc: 0.6678
Epoch 4/100
2652/2652 [==============================] - 8s 3ms/step - loss: 0.0038 - tp: 135.0000 - fp: 35.0000 - tn: 84654.0000 - fn: 32.0000 - accuracy: 0.9992 - precision: 0.7941 - recall: 0.8084 - auc: 0.9454 - prc: 0.7199
Epoch 5/100
2652/2652 [==============================] - 7s 3ms/step - loss: 0.0037

In [73]:
import warnings
warnings.filterwarnings("ignore")

In [74]:
# Evaluate the model using the test data
baseline_results = nn_model.evaluate(X_test,y_test,verbose=2)

884/884 - 2s - loss: nan - tp: 54.0000 - fp: 4.0000 - tn: 28207.0000 - fn: 21.0000 - accuracy: 0.9991 - precision: 0.9310 - recall: 0.7200 - auc: 0.8931 - prc: 0.7788 - 2s/epoch - 3ms/step
